# Tutorial on `opr.inference.pipelines.place_recognition_pipeline` subpackage

This tutorial demonstrates the new top‑k Place Recognition pipeline that uses the `Index` module.

You will:
- Build a tiny on‑disk database index: `descriptors.npy`, `meta.parquet` (with `idx`, `pose[7]`), `schema.json`.
- Create a stub model that returns a descriptor (as if it was computed by a real network).
- Load the FAISS Flat index, run a top‑k search via `PlaceRecognitionPipeline`, and interpret the outputs.

Requirements:
- `faiss` (faiss‑cpu or faiss‑gpu)
- `pandas` + a Parquet engine: `pyarrow` (recommended) or `fastparquet`
- `torch`


In [1]:
# Install/check requirements (optional)
import importlib
missing = []
for pkg in ("faiss", "pandas", "numpy", "pyarrow", "torch"):
    if importlib.util.find_spec(pkg) is None:
        missing.append(pkg)
if missing:
    print("Missing packages:", ", ".join(missing))
    print("Install with: pip install faiss-cpu pyarrow pandas numpy torch")
else:
    print("Environment looks good.")


Environment looks good.


In [2]:
# Prepare a tiny database index
from pathlib import Path
import json
import numpy as np
import pandas as pd

base = Path("./_demo_index_pr").resolve()
base.mkdir(parents=True, exist_ok=True)

N, D = 10, 8
rng = np.random.default_rng(1)
descriptors = rng.normal(size=(N, D)).astype(np.float32)
poses = [[float(i), float(i+1), float(i+2), 0.0, 0.0, 0.0, 1.0] for i in range(N)]
meta = pd.DataFrame({"idx": np.arange(1000, 1000+N, dtype=np.int64), "pose": poses})

np.save(base / "descriptors.npy", descriptors)
meta.to_parquet(base / "meta.parquet")
schema = {"version": "1", "dim": D, "metric": "l2", "created_at": "", "opr_version": ""}
(base / "schema.json").write_text(json.dumps(schema))

print(f"Wrote: {[p.name for p in base.iterdir()]}")


Wrote: ['meta.parquet', 'descriptors.npy', 'schema.json']


In [3]:
# Create a stub model that returns a query descriptor
import torch
from torch import nn

class StubModel(nn.Module):
    def __init__(self, descriptor: np.ndarray):
        super().__init__()
        self.register_buffer("_desc", torch.from_numpy(descriptor.astype(np.float32, copy=False)))
    def forward(self, _: dict[str, torch.Tensor]):
        return {"final_descriptor": self._desc.unsqueeze(0)}

query_desc = descriptors[0] + 0.02
model = StubModel(query_desc)
print(f"Stub model ready, descriptor norm: {float(torch.linalg.norm(model._desc))}")


Stub model ready, descriptor norm: 2.0769312381744385


In [ ]:
# Load index and run PlaceRecognitionPipeline
from opr.inference.index import FaissFlatIndex
from opr.inference.pipelines import PlaceRecognitionPipeline

index = FaissFlatIndex.load(base)
pipeline = PlaceRecognitionPipeline(index=index, model=model, device="cpu")

result = pipeline.infer(input_data={}, k=5)
print("descriptor shape:", result.descriptor.shape)
print("indices:", result.indices.tolist())
print("distances:", [float(x) for x in result.distances])
print("db_idx:", result.db_idx.tolist())
print("db_pose (first):", result.db_pose[0])
